In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
file_path = '/content/drive/MyDrive/Data/london_preprocessing.csv'
df = pd.read_csv(file_path)

In [4]:
df.drop(columns=['is_english'], inplace=True)

In [5]:
df.drop(columns=['id'], inplace=True)

In [6]:
df.drop(columns=['reviewer_id'], inplace=True)

In [7]:
df.head()

,listing_id,date,reviewer_name,comments
0,13913,2022-09-16,Ye,Alina is a very generous and warm-hearted host...
1,13913,2022-09-20,Violeta,Cancelled on the same day I arrived to London
2,13913,2022-09-22,Alexander,Great flat for a short stay with everything yo...
3,13913,2022-09-23,Robert,"Great stay again, Alina is friendly and welcom..."
4,13913,2022-09-26,Devaraj,I had an excellent stay in Alina's place


In [8]:
# Import necessary libraries
import pandas as pd                   # For data manipulation
import matplotlib.pyplot as plt       # For plotting
import re                             # For regular expressions
from pprint import pprint             # For pretty-printing

# Gensim: A topic modeling library
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spaCy: A natural language processing library for lemmatization
import spacy

# Visualization tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt       # Matplotlib for additional plotting
%matplotlib inline

# Enable logging for Gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Ignore deprecation warnings during execution
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)



In [9]:
pip install pyLDAvis

In [10]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Import the stopwords module from NLTK (Natural Language Toolkit)
from nltk.corpus import stopwords

# Retrieve the set of English stopwords
stop_words = stopwords.words('english')


In [12]:
# Define a function to tokenize sentences into words using Gensim's simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        # Yield a list of lowercase words after simple preprocessing (removing accents)
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Apply the tokenization function to the 'comments' column of the DataFrame
data_words = list(sent_to_words(df['comments']))

# Print the tokenized words for the first sentence in the DataFrame
print(data_words[:1])


[['alina', 'is', 'very', 'generous', 'and', 'warm', 'hearted', 'host', 'staying', 'with', 'her', 'is', 'very', 'chill', 'and', 'relaxed', 'will', 'feel', 'just', 'like', 'home', 'strongly', 'recommended', 'especially', 'if', 'are', 'traveling', 'alone']]


In [13]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['alina', 'is', 'very', 'generous', 'and', 'warm', 'hearted', 'host', 'staying', 'with', 'her', 'is', 'very', 'chill', 'and', 'relaxed', 'will', 'feel', 'just', 'like', 'home', 'strongly', 'recommended', 'especially', 'if', 'are', 'traveling', 'alone']


In [14]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spaCy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spaCy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Perform lemmatization keeping noun, adjective, verb, and adverb
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['generous', 'warm', 'hearted', 'host', 'stay', 'chill', 'relaxed', 'feel', 'home', 'strongly', 'recommend', 'especially', 'travel', 'alone']]


In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [17]:
id2word[0]

'alone'

In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alone', 1),
  ('chill', 1),
  ('especially', 1),
  ('feel', 1),
  ('generous', 1),
  ('hearted', 1),
  ('home', 1),
  ('host', 1),
  ('recommend', 1),
  ('relaxed', 1),
  ('stay', 1),
  ('strongly', 1),
  ('travel', 1),
  ('warm', 1)]]

In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.188*"beautiful" + 0.118*"short" + 0.071*"view" + 0.066*"modern" + '
  '0.058*"garden" + 0.052*"early" + 0.047*"neighbourhood" + 0.044*"touch" + '
  '0.039*"recommendation" + 0.029*"whole"'),
 (1,
  '0.050*"time" + 0.033*"br" + 0.032*"go" + 0.031*"book" + 0.028*"house" + '
  '0.027*"look" + 0.027*"experience" + 0.026*"even" + 0.025*"much" + '
  '0.024*"provide"'),
 (2,
  '0.116*"make" + 0.109*"feel" + 0.103*"home" + 0.053*"right" + '
  '0.048*"welcome" + 0.033*"tidy" + 0.028*"safe" + 0.027*"sure" + '
  '0.026*"coffee" + 0.024*"comfy"'),
 (3,
  '0.103*"people" + 0.064*"problem" + 0.050*"cosy" + 0.049*"ideal" + '
  '0.046*"communicate" + 0.046*"toilet" + 0.041*"noisy" + 0.034*"facility" + '
  '0.030*"service" + 0.028*"care"'),
 (4,
  '0.045*"work" + 0.043*"use" + 0.040*"overall" + 0.034*"park" + 0.029*"door" '
  '+ 0.029*"shower" + 0.024*"guest" + 0.022*"towel" + 0.021*"water" + '
  '0.020*"key"'),
 (5,
  '0.127*"walk" + 0.110*"station" + 0.071*"minute" + 0.049*"property" + '
  

In [26]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.430821670848125

Coherence Score:  0.5891107436248401


In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
visualization = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.310817 -0.113328       1        1  39.250814
1      0.130482 -0.073639       2        1  11.047186
6      0.113061 -0.069703       3        1  10.510515
11     0.091927 -0.073314       4        1   9.651817
5      0.043624  0.316606       5        1   6.492246
4      0.036550  0.312124       6        1   6.274555
2      0.000930 -0.059886       7        1   5.687512
10    -0.105542 -0.052565       8        1   3.210798
7     -0.142309 -0.049128       9        1   2.301354
0     -0.142362 -0.049179      10        1   2.240906
3     -0.156445 -0.047497      11        1   2.066308
9     -0.180734 -0.040490      12        1   1.265989, topic_info=           Term           Freq          Total Category  logprob  loglift
10         stay  270899.000000  270899.000000  Default  30.0000  30.0000
20        great  205457.000000  205457.000000  Default  29.0000  29.0000
29         room   80097.000000   80097.000000  Default  28.0000  28.0000
65         walk   65980.000000   65980.000000  Default  27.0000  27.0000
32        place  189204.000000  189204.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
315        rule     862.212908     863.195864  Topic12  -4.7673   4.3682
1548  sensitive     772.913673     773.896622  Topic12  -4.8767   4.3680
366        heat     720.841973     721.824908  Topic12  -4.9464   4.3680
3558      roomy     569.680005     570.662938  Topic12  -5.1818   4.3676
2557       roof     569.017677     570.000620  Topic12  -5.1829   4.3676

[391 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
550       3  0.999879          able
595       3  0.999955    absolutely
67        3  0.999941        access
1740     10  0.999022    accomodate
567      10  0.999408  accomodation
...     ...       ...           ...
1408     10  0.999856         whole
499       3  0.999948        window
104       6  0.999960          work
602       5  0.999910         worth
234      12  0.999138          zone

[362 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 7, 12, 6, 5, 3, 11, 8, 1, 4, 10])

In [29]:
# Export the visualization as a html file.
pyLDAvis.save_html(visualization, 'drive/My Drive/Colab Notebooks/LDAModel12.html')